In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
#import necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau
import cv2
import os
from glob import glob
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [23]:
#downloading data from folder
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train'
test_dir =  '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test'
val_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val'

In [42]:
#Define function that will help split our data into X and Y and resize it 
def picture_separation(folder):
    """
    source code from:https://medium.com/analytics-vidhya/how-to-load-any-image-dataset-in-python-3bd2fa2cb43d
    and https://www.kaggle.com/rafetcan/
    input: folder
    """
    y = []
    x = []
    image_list = []

    for foldername in os.listdir(folder):
        if not foldername.startswith('.'):
            if foldername == "NORMAL":
                label = 0
            elif foldername == "PNEUMONIA":
                label = 1
            else:
                label = 2
                
            for image_filename in os.listdir(folder + "/"+ foldername):
                img_file = cv2.imread(folder + "/" + foldername + '/' + image_filename,0)               
                if img_file is not None:
                    img = cv2.resize(img_file,(64,64)) #resize img
                    img_arr = img_to_array(img) / 255
                    x.append(img_arr)
                    y.append(label)
                    image_list.append(foldername + '/' + image_filename)
                                        
    X = np.asarray(x) #create an array of images for X
    y = np.asarray(y) #create an array of images for y
    
    

    return X,y,image_list


In [25]:
#put the images in a dataframe 
X_train, y_train, img_train = picture_separation(train_dir)

train_df = pd.DataFrame(img_train, columns=["images"])
train_df["target"] = y_train

In [26]:
#checking the shape of the X_train 
X_train.shape

In [27]:
#checking the shape of the y_train 
y_train.shape

In [28]:
#check the first 4 rows of the train dataset
train_df.head()

In [29]:
#check the info of the train dataset
train_df.info()

In [30]:
#put the val dataset in a dataframe 
X_val, y_val, img_val = picture_separation(val_dir)

val_df = pd.DataFrame(img_val, columns=["images"])
val_df["target"] = y_val

In [31]:
X_val.shape

In [32]:
#put the test dataset in a dataframe 
X_test, y_test, img_test = picture_separation(test_dir)

test_df = pd.DataFrame(img_test, columns=["images"])
test_df["target"] = y_test

In [33]:
X_test.shape

In [34]:
#concat the dataset into a dataframe 
#so that we can see the shape and the info
full_data = pd.concat([train_df, test_df, val_df], axis=0, ignore_index=True)

In [35]:
#view the first four rows and the last five rows
print(full_data.head())
print(full_data.tail())

In [36]:
full_data.shape

In [37]:
#view the full dataset info
full_data.info()

In [38]:
#checking if the target is balance 
sns.countplot(full_data["target"])
plt.title("NORMAL/PNOMONİA")
plt.show()
print(full_data["target"].value_counts())

**Observation** 
- Clearly, the data seems to be imbalanced in this case. Now in order to balance the distribution of the training examples, we would be using data augmentation procedure. first let's visualize the images to see the X-ray that is Normal and not Normal(Pneumonia).


In [39]:
#plot the images that is normal and Pneumonia
plt.figure(figsize=(15,8))

plt.subplot(2,3,1) 
img = load_img(train_dir + "/" + full_data["images"][0])
plt.imshow(img)
plt.title("PNEUMONIA", color = "blue", size = 14)
plt.axis("off")


plt.subplot(2,3,2) 
img = load_img(train_dir + "/" + full_data["images"][1])
plt.imshow(img)
plt.title("PNEUMONIA", color = "blue", size = 14)
plt.axis("off")

plt.subplot(2,3,3) 
img = load_img(train_dir + "/" + full_data["images"][10])
plt.imshow(img)
plt.title("PNEUMONIA", color = "blue", size = 14)
plt.axis("off")

plt.subplot(2,3,4) 
img = load_img(train_dir + "/" + full_data["images"][3875])
plt.imshow(img)
plt.title("NORMAL", color = "green", size = 14)
plt.axis("off")

plt.subplot(2,3,5) 
img = load_img(train_dir + "/" + full_data["images"][3876])
plt.imshow(img)
plt.title("NORMAL", color = "green", size = 14)
plt.axis("off")

plt.subplot(2,3,6) 
img = load_img(train_dir + "/" + full_data["images"][3877])
plt.imshow(img)
plt.title("NORMAL", color = "green", size = 14)
plt.axis("off")

plt.suptitle("NORMAL or PNEUMONIA", size = 16, color = "darkred")
plt.show()

# Data Augmentation
- Data augmentation is used to increase the size of our data based on the existing data. it acts asa regularizer to help reduce overfitting, especially in this case where we have an imbalanced target.

In [116]:
# Fitting the CNN to the images
# The function ImageDataGenerator augments your image by iterating through image as your CNN is getting ready to process that image

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.3,
                                   brightness_range=(0.9,1),
                                   zoom_range = 0.3,
                                   validation_split= 0.1)

test_datagen = ImageDataGenerator(rescale = 1./255)  #Image normalization.

val_datagen = ImageDataGenerator(rescale = 1./255) #Image normalization.

training_generator = train_datagen.flow_from_directory(train_dir,
                                                       target_size = (64, 64),
                                                       batch_size = 32,
                                                       subset = 'training',
                                                       color_mode='grayscale',
                                                       class_mode = 'binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(64, 64),
                                                  batch_size=32,
                                                  color_mode='grayscale',
                                                  class_mode='binary')

val_generator = train_datagen.flow_from_directory(train_dir,
                                                  target_size = (64, 64),
                                                  batch_size = 32,
                                                  class_mode = 'binary',
                                                  color_mode='grayscale',
                                                  subset ='validation')

In [122]:
#define an input shape so that we can use it for modeling
input_shape = X_train.shape[1:]
input_shape

In [137]:
#Instantiate sequential and add input layer
model = Sequential()
model.add(Conv2D(32,(3,3),activation = 'relu',padding='same',input_shape = input_shape))
model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D())

#Layers
model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D())

#layers
model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D())

#output
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer ='adam',
             metrics=['accuracy'])


In [138]:
#get the model summary
model.summary()

In [139]:
#we are creating a early stopping so that when our 
#model reaches the desired level we can stop it
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

In [140]:
#examine the accuracy score and the loss score using the holdout training validation set.
history = model.fit_generator(training_generator,
                             steps_per_epoch=4695//32,
                             epochs =20,
                             validation_data = val_generator,
                             validation_steps= 521//32,
                             callbacks=[early_stopping])

In [150]:
#evaluate the model
print('Acuracy of the model is: ',model.evaluate_generator(val_generator)[1]*100,'%')
print('Loss of the model is: ', model.evaluate_generator(val_generator)[0])

In [146]:
history.history.keys()

In [151]:
#plot the accuracy and loss on a graph to visualize how the model is doing.
plt.figure()
plt.plot(history.history["loss"],label = "Train Loss")
plt.plot(history.history["val_loss"],label = "Validation Loss")
plt.legend()
plt.show()

plt.figure()
plt.plot(history.history["accuracy"],label = "Train Accuracy")
plt.plot(history.history["val_accuracy"],label = "Validation Accuracy")
plt.legend()
plt.show()